import ipotrant libraries

In [1]:
import ROOT
import math
from array import array
import numpy as np
from ROOT import gROOT, AddressOf
%jsroot on

Welcome to JupyROOT 6.29/01


Set common variables

In [2]:
read_ampt_dat_files = False

ampt_files_path = "/home/yoren/bnl/AMPT/Artem_heau/output_taxi/"

N_files = 200

Define structures

In [3]:
gROOT.ProcessLine(
"struct event {\
  Int_t           number;\
  Int_t         ncharge_y;\
  Float_t         phi_bbc;\
  Float_t         phi_cnt;\
  Float_t         phi_fvtx;\
  };"
); 


class event:
  
  def __init__(self, input_array):
    self.number = int(input_array[0])-100
    self.check = int(input_array[1])
    self.nch_tot = int(input_array[2])
    self.bimp = float(input_array[3])
    self.npart1 = int(input_array[4])
    self.npart2 = int(input_array[5])
    self.ncoll_ell = int(input_array[6])
    self.ncoll = int(input_array[7])
    self.ncoll1 = int(input_array[8])
    self.ncharge_y = int(input_array[9])
    self.phi_bbc = float(input_array[10])
    self.phi_cnt = float(input_array[11])
    self.phi_fvtx = float(input_array[12])
  
  
  def __str__(self):
    return f"number = {self.number}, check = {self.check}, nch_tot = {self.nch_tot}, bimp = {self.bimp}, npart1 = {self.npart1}, npart2 = {self.npart2}, "+f"ncoll_ell = {self.ncoll_ell}, ncoll = {self.ncoll}, ncoll1 = {self.ncoll1}, ncharge_y = {self.ncharge_y}, phi_bbc = {self.phi_bbc}, phi_cnt = {self.phi_cnt}, phi_fvtx = {self.phi_fvtx},"
  
  def get_values(self):
    list = [self.number, self.ncharge_y, self.phi_bbc, self.phi_cnt, self.phi_fvtx]
    for i in range(len(list)):
      if list[i]>1e6 or list[i]<-5:
        list[i]=9999
    #arr = np.array(list)
    return list
  
  def set_event(self, mystruck):
    mystruck.number = self.number
    mystruck.ncharge_y = self.ncharge_y
    mystruck.phi_bbc = self.phi_bbc
    mystruck.phi_cnt = self.phi_cnt
    mystruck.phi_fvtx = self.phi_fvtx

  
class particles:
  def __init__(self):
    self.px_pi0 = []
    self.py_pi0 = []
    self.pz_pi0 = []
    self.m_pi0 = []
    self.px = []
    self.py = []
    self.pz = []
    self.m = []

  def __str__(self):
    return f"N_pi0 = {len(self.m_pi0)}, N_ch_part = {len(self.m)}"
  
  def get_Nums(self):
    return len(self.m), len(self.m_pi0)

  def add_charged_particle(self, input_mom):
    self.px.append(float(input_mom[0]))
    self.py.append(float(input_mom[1]))
    self.pz.append(float(input_mom[2]))
    self.m.append(float(input_mom[3]))

  def add_pi0(self, input_mom):
    self.px_pi0.append(float(input_mom[0]))
    self.py_pi0.append(float(input_mom[1]))
    self.pz_pi0.append(float(input_mom[2]))
    self.m_pi0.append(float(input_mom[3]))

Create TTree creator

In [4]:
event_input = [array('i', [0]), array('i', [0]), array('d', [0]), array('d', [0]), array('d', [0]), array('i', [0]), array('i', [0])]
hadron_input = [array('d', [0]), array('d', [0]), array('d', [0]), array('d', [0])]
pi0_input = [array('d', [0]), array('d', [0]), array('d', [0]), array('d', [0])]
num_input1, num_input2 = array('i', [0]), array('i', [0])
def ttree_creator(name1, name2, name3):
    tree_event0 = ROOT.TTree(name1,name1)
    tree_event0.Branch('number', event_input[0], 'number/I')
    tree_event0.Branch('nhadrons', event_input[5], 'nhadrons/I')
    tree_event0.Branch('npi0s', event_input[6], 'npi0s/I')
    tree_event0.Branch('ncharge_y', event_input[1], 'ncharge_y/I')
    tree_event0.Branch('phi_bbc', event_input[2], 'phi_bbc/D')
    tree_event0.Branch('phi_cnt', event_input[3], 'phi_cnt/D')
    tree_event0.Branch('phi_fvtx', event_input[4], 'phi_fvtx/D')


    tree_hadron0 = ROOT.TTree(name2,name2)

    tree_hadron0.Branch('number', num_input1, 'number/I')

    tree_hadron0.Branch("px", hadron_input[0], 'px/D')
    tree_hadron0.Branch("py", hadron_input[1], 'py/D')
    tree_hadron0.Branch("pz", hadron_input[2], 'pz/D')
    tree_hadron0.Branch("m",  hadron_input[3],  'm/D')\
    
    tree_pi00 = ROOT.TTree(name3,name3)

    tree_pi00.Branch('number', num_input2, 'number/I')

    tree_pi00.Branch("px", pi0_input[0], 'px/D')
    tree_pi00.Branch("py", pi0_input[1], 'py/D')
    tree_pi00.Branch("pz", pi0_input[2], 'pz/D')
    tree_pi00.Branch("m",  pi0_input[3],  'm/D')
    
    return [tree_event0, tree_hadron0, tree_pi00]



    #mystruck= ROOT.event()
    #tree.Branch("event",mystruck,"number/I:ncharge_y/I:phi_bbc/F:phi_cnt/F:phi_fvtx/F")
    #N=5
    #npa = np.array(N*[ 0.2 ])
    #tree.Branch('nparrayb', npa, 'nparrayb[' + str(N) + ']/D')
    #tree.Branch("point",event,"number:check:nch_tot/i:bimp/F:npart1:npart2:ncoll_ell:ncoll:ncoll1:ncharge_y/I:phi_bbc:phi_cnt:phi_fvtx/F")


Create file reader

In [5]:
def file_reader(filename, tree_event_in, tree_hadron_in, tree_pi0_in):
    f = open(filename, "r")
    part = particles()
    for line in f:
        line = line.split()
        if float(line[0]) > 100:
            event_input[5][0], event_input[6][0] = part.get_Nums()
            if float(line[0]) > 101: tree_event_in.Fill()  
            test = event(line)
            npa = test.get_values()
            for i in range(5):
                event_input[i][0] = npa[i]
            part = particles()
            num_input1[0], num_input2[0] = npa[0], npa[0]
        else :
            if float(line[3]) == 0.1350: 
                part.add_pi0(line)
                for i in range(4):
                    pi0_input[i][0] = float(line[i])    
                tree_pi0_in.Fill()            
            else: 
                part.add_charged_particle(line)
                for i in range(4):
                    hadron_input[i][0] = float(line[i])    
                tree_hadron_in.Fill()
    event_input[5][0], event_input[6][0] = part.get_Nums()
    tree_event_in.Fill()
    
    #for i in range(len(part.px)):
        #e_input[0][i]=part.px[i]
    #tree.Fill()


Read AMPT files and create TTree

In [6]:
[tree_event, tree_hadron, tree_pi0] = ttree_creator("event","hadron","pi0")
if read_ampt_dat_files:  
    for i in range(1,1+N_files):
        filename = ampt_files_path+f"ampt_{i}.dat"
        file_reader(filename, tree_event, tree_hadron, tree_pi0)

tree_event.Print()
tree_hadron.Print()
tree_pi0.Print()


******************************************************************************
*Tree    :event     : event                                                  *
*Entries :        0 : Total =            3813 bytes  File  Size =          0 *
*        :          : Tree compression factor =   1.00                       *
******************************************************************************
*Br    0 :number    : number/I                                               *
*Entries :        0 : Total  Size=        489 bytes  One basket in memory    *
*Baskets :        0 : Basket Size=      32000 bytes  Compression=   1.00     *
*............................................................................*
*Br    1 :nhadrons  : nhadrons/I                                             *
*Entries :        0 : Total  Size=        497 bytes  One basket in memory    *
*Baskets :        0 : Basket Size=      32000 bytes  Compression=   1.00     *
*...................................................

Write Tree to RootFile

In [7]:
if read_ampt_dat_files:
  trees=[tree_event,tree_hadron,tree_pi0]
  output_file = ROOT.TFile.Open("HeAu_Artem_new.root","RECREATE")
  for tree in trees: tree.Write()
  output_file.Close()

Read TTree

In [8]:
tree_read_event, tree_read_hadron, tree_read_pi0 = tree_event, tree_hadron, tree_pi0

inFile = ROOT.TFile.Open('HeAu_Artem_new.root',"read")

tree_read_event = inFile.Get("event")
tree_read_pi0 = inFile.Get("pi0")
tree_read_hadron = inFile.Get("hadron")

"""for ev in tree_read_event:
     print(ev.npi0s)
     if ev.npi0s==10: break
for ev in tree_read_pi0:
    if ev.m != 0.1350: print(ev.m)
a=0
for ev in tree_read_hadron:
    if ev.m == 0.0005: a+=1
    if a==10: break
print(a)"""

inFile.Close()


In [9]:
random = ROOT.gRandom.Rndm

def Sphere(r):
    a, b,r2 = 0, 0, 1
    while r2 > 0.25:
        a  = random() - 0.5
        b  = random() - 0.5
        r2 =  a*a + b*b
    z = r* ( -1. + 8.0 * r2 )
 
    scale = 8.0 * r * (0.25 - r2)**0.5
    x = a*scale
    y = b*scale

    return [r, x, y, z]


In [10]:
def lorenz_boost(pt_origin, m=0.547):

    p_tot=(pt_origin[0]**2+pt_origin[1]**2+pt_origin[2]**2)**0.5
    p0=[(m**2+p_tot**2)**0.5,pt_origin[0],pt_origin[1],pt_origin[2]]
    p0=np.array(p0)
    
    gamma = p0[0]/m
    beta = p_tot/p0[0]
    beta_x, beta_y, beta_z = -p0[1]/p0[0], -p0[2]/p0[0], -p0[3]/p0[0]
#print(p0, gamma, beta, beta_x, beta_y, beta_z)
    E1, E2 = m/2, m/2
    p1 = Sphere(E1)
    p2 = [E2,-p1[1],-p1[2],-p1[3]]
    p1=np.array(p1)
    p2=np.array(p2)

    #print(p0, p1,p2)
    boost_matrix = np.matrix(f"{gamma} {-gamma*beta_x} {-gamma*beta_y} {-gamma*beta_z};\
                            {-gamma*beta_x} {1+(gamma-1)*(beta_x/beta)**2} {(gamma-1)*beta_x*beta_y/beta**2} {(gamma-1)*beta_x*beta_z/beta**2};\
                            {-gamma*beta_y} {(gamma-1)*beta_x*beta_y/beta**2} {1+(gamma-1)*(beta_y/beta)**2} {(gamma-1)*beta_y*beta_z/beta**2};\
                            {-gamma*beta_z} {(gamma-1)*beta_x*beta_z/beta**2} {(gamma-1)*beta_z*beta_y/beta**2} {1+(gamma-1)*(beta_z/beta)**2}")
    
    p3 = np.dot(boost_matrix,p1)
    p4 = np.dot(boost_matrix,p2)

    p3 = [p3[0,0],p3[0,1],p3[0,2],p3[0,3]]
    p4 = [p4[0,0],p4[0,1],p4[0,2],p4[0,3]]

    return p3, p4
    
    #print(boost_matrix)
    #print(p3,p4)
    #print(p3[0,0]**2-p3[0,1]**2-p3[0,2]**2-p3[0,3]**2)
    #for i in range(4):
        #print(p0[i]-p3[0,i]-p4[0,i])

#x,y = lorenz_boost([2,1,0.5])
#print(x,y)

In [11]:
import math

def angle(vector1, vector2):
    x1, y1, z1 = vector1[1], vector1[2], vector1[3]
    x2, y2, z2 = vector2[1], vector2[2], vector2[3]
    inner_product = x1*x2 + y1*y2 + z1*z2
    len1 = math.hypot(x1, y1, z1)
    len2 = math.hypot(x2, y2, z2)
    #print(inner_product,len1,len2)
    rel = inner_product/(len1*len2)
    if abs(rel)>1:
        rel = 1
    return math.acos(rel)

veto_angle = math.atan(20./522)
print(veto_angle)

0.03829544464261488


In [12]:
def inv_mass(vector1, vector2):
    e1, px1, py1, pz1 = vector1[0], vector1[1], vector1[2], vector1[3]
    e2, px2, py2, pz2 = vector2[0], vector2[1], vector2[2], vector2[3]
    m2 = (e1+e2)**2 - (px1+px2)**2 - (py1+py2)**2 - (pz1+pz2)**2
    if m2<0: m2=0
    return math.sqrt(m2)

In [13]:

hist_eta = ROOT.TH1D("hist_eta","hist_eta",100,0,10)
veto_eta = ROOT.TH1D("veto_eta","veto_eta",100,0,10)


inFile = ROOT.TFile.Open('HeAu_Artem_new.root',"read")

tree_read_event = inFile.Get("event")
tree_read_pi0 = inFile.Get("pi0")
tree_read_hadron = inFile.Get("hadron")

Nev = tree_read_event.GetEntries()
hadron_input = [array('d', [0]), array('d', [0]), array('d', [0]), array('d', [0])]
pi0_input = [array('d', [0]), array('d', [0]), array('d', [0]), array('d', [0])]
num_inputs = [array('i', [0]), array('i', [0])]
tree_read_hadron.SetBranchAddress('number', num_inputs[0])
tree_read_pi0.SetBranchAddress('number', num_inputs[1])
tree_read_hadron.SetBranchAddress('px', hadron_input[0])
tree_read_hadron.SetBranchAddress('py', hadron_input[1])
tree_read_hadron.SetBranchAddress('pz', hadron_input[2])
tree_read_pi0.SetBranchAddress('px', pi0_input[0])
tree_read_pi0.SetBranchAddress('py', pi0_input[1])
tree_read_pi0.SetBranchAddress('pz', pi0_input[2])

print(tree_read_event.GetEntries())

ievent = 0
ih_init = 0
ipi0_init = 0
for ev in tree_read_event:
    
    npi0s = ev.npi0s
    nhadrons = ev.nhadrons
    ievent0=ev.number

    hadrons_in_event=[]
    for ihadron in range(ih_init,ih_init+nhadrons):
        tree_read_hadron.GetEntry(ihadron)
        pt = (hadron_input[0][0]**2+hadron_input[1][0]**2)**0.5
        if pt<0.1: continue
        hadrons_in_event.append([0,hadron_input[0][0],hadron_input[1][0],hadron_input[2][0]])

    ih_init+=nhadrons

    for ipi0 in range(ipi0_init,ipi0_init+npi0s):
        tree_read_pi0.GetEntry(ipi0)
        p1, p2 = lorenz_boost([pi0_input[0][0],pi0_input[1][0],pi0_input[2][0]])
        pi0_pt = (pi0_input[0][0]**2+pi0_input[1][0]**2)**0.5
        if pi0_pt<0.099: continue
        hist_eta.Fill(pi0_pt)
        flag = False
        for hadron in hadrons_in_event:
            if angle(hadron, p1)<veto_angle or angle(hadron, p2)<veto_angle: flag = True
        if flag: continue
        veto_eta.Fill(pi0_pt)
        
    ipi0_init+=npi0s
    
    if ievent > 500000: break
    ievent+=1

inFile.Close()

500000


In [26]:
ratio = ROOT.TH1D("ratio","ratio",100,0,10)
divider = ROOT.TH1D("divider","divider",100,0,10)

c1 = ROOT.TCanvas("c1","c1",720*2,720)
c1.Divide(2,1)

c1.cd(1)
hist_eta.Draw()
veto_eta.Draw("same")
veto_eta.SetLineColor(2)

c1.cd(2)
ratio.Add(veto_eta)
divider.Add(hist_eta)
ratio.Rebin(3)
divider.Rebin(3)
ratio.Divide(divider)
#ratio.Scale(1)
ratio.Draw("p")
ratio.SetMarkerStyle(21)
ratio.SetMarkerColor(6)

c1.Draw()
c1.SaveAs("output/kek.png")

Warning in <TROOT::Append>: Replacing existing TH1: ratio (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: divider (Potential memory leak).
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1
Warning in <TH1D::Rebin>: ngroup=3 is not an exact divider of nbins=100.
Warning in <TH1D::Rebin>: ngroup=3 is not an exact divider of nbins=100.
Info in <TCanvas::Print>: png file output/kek.png has been created


In [17]:
hists=[hist_eta, veto_eta, ratio]
output_file = ROOT.TFile.Open("output/HeAu_Artem_hists.root","RECREATE")
for hist in hists: hist.Write()
output_file.Close()

In [15]:
#second(156,5000)
import multiprocess as mp

Ntr = nThread
Nev = nEvents
seed0 = 15156
seed_iter = 43324

pn_ids = [int(2212), int(2112)]

projectile_params = [pn_ids[int(1.34*i/(Ntr))] for i in range(Ntr)]
target_params = [pn_ids[min(int(4*i/Ntr), 1)] for i in range(Ntr)]
print(projectile_params, target_params)

pool = mp.Pool(Ntr)
hist_array = pool.starmap(pythia8_runner, [(seed0+i*seed_iter, Nev, 2212, 2212) for i in range(Ntr)])
#hist_array = pool.starmap(pythia8_runner, [(seed0+i*seed_iter, Nev, 1000020040, 1000791970) for i in range(Ntr)]) #1000020030
#hist_array = pool.starmap(pythia8_runner, [(seed0+i*seed_iter, Nev, projectile_params[i], target_params[i]) for i in range(Ntr)])
pool.close()

NameError: name 'nThread' is not defined

In [ ]:
hist_Nch = hist_array[0][0]
for i in range(1, Ntr):
    hist_Nch.Add(hist_array[i][0])
hist_pt = hist_array[0][1]
for i in range(1, Ntr):
    hist_pt.Add(hist_array[i][1])
hist_imass = hist_array[0][2]
for i in range(1, Ntr):
    hist_imass.Add(hist_array[i][2])
hist_Nch_mid = hist_array[0][3]
for i in range(1, Ntr):
    hist_Nch_mid.Add(hist_array[i][3])

NameError: name 'hist_array' is not defined